In [24]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt

In [27]:
#solving the wave equation: f_tt=c^2*f_xx
N=100; # number of grid boxes
L=1; # length of the domain
c=1; #speed of waves.
x=np.linspace(-L/2,L/2,N); 
dx=x[1]-x[0];

In [28]:
# function f_initial defines the initial conditions

def f_initial(x,L):
    f0= np.exp(-(x/(L/20))**2/2)*np.cos(2*np.pi*x/(L/10)); #gaussian * cos is a localized wave
    return f0


f0=f_initial(x,L);
f1=np.zeros(len(f0));
f=f0;

# Courant stability requires C<1
dt=0.5*dx/c;
total_time=1; #simulation time
Nt=np.fix(total_time/dt); # number of timesteps
f_xx=np.zeros(len(f0));



f_theory = f0

fig,ax = plt.subplots()

line1 = ax.plot(x,f,'-k',label='Simulation')[0]
line2 = ax.plot(x,f_theory,'-r',label='Theory')[0]
f_prev=f
plt.legend()

#plotting

def update(frame):
    global f,f_theory,f_prev
    time = frame*dt
    x_shift_left=x+c*time; 

    x_shift_left=np.remainder(x_shift_left+L/2,L)-L/2; 
    # shifted x in the positive direction will exceed the domain and hence needs to be put back into the domain; 
    # this is for periodic boundary conditions only

    x_shift_right=x-c*time; 

    x_shift_right=np.remainder(x_shift_right-L/2,L)-L/2;

    f_theory=0.5*f_initial(x_shift_right,L) + 0.5*f_initial(x_shift_left,L);
    #theoretical solution, works only for periodic boundary conditions (or an infinite domain)
    
    # calculating f_xx using a simple central difference scheme, O(dx^2)
    f_xx[1:-1]=(f[2:]-2*f[1:-1] + f[:-2])/dx**2;
    
    f_xx[0] = (f[1] - 2 * f[0] + -f[0])/dx**2 #left boundary
    f_xx[-1]= (f[-1] - 2 * f[-1] + f[-2])/dx**2; #right boundary
    
    if frame==0:
        f_next=f0+f1*dt + 0.5*dt**2*c**2*f_xx;
    else:
        f_next=2*f- f_prev + dt**2*(c**2*f_xx);
    f_prev=f; #save the current value to use as the previous value during the next time step
    f=f_next; #update the current value for this timestep.
    
    line1.set_ydata(f)
    line2.set_ydata(f_theory)
    return line1,line2


ani = FuncAnimation(fig,update,frames=int(Nt),interval=25,repeat=False)